# Ch12 并发编程(Concurrency)

In [9]:
import time
def countdown(n):
    while n > 0:
        print('T - minus', n)
        n -=  1
        time.sleep(5)
        
from threading import Thread
t = Thread(target=countdown, args=(10, ), daemon=True)
t.start()

T - minus 10
T - minus 9
T - minus 8
T - minus 7
T - minus 6
T - minus 5
T - minus 4
T - minus 3
T - minus 2
T - minus 1


In [8]:
if t.is_alive():
    print('Still runing')
else:
    print('Completed')

Still runing
T - minus 7
T - minus 6
T - minus 5
T - minus 4
T - minus 3
T - minus 2
T - minus 1


In [15]:
class CountdownTask:
    def __init__(self):
        self._running = True
        
    def terminate(self):
        self._running = False
        
    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)
            
c = CountdownTask()
t = Thread(target=c.run, args=(10, ))
t.start()
#c.terminate()
# t.join()

T-minus 10
T-minus 9
T-minus 8
T-minus 7


In [17]:
t.join()

In [18]:
class IOTask:
    def terminate(self):
        self_running = Flase
        
    def run(self, sock):
        sock.settimeout(5)
        while self._running:
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
                

In [19]:
from threading import Thread

class CountdownThread(Thread):
    def __init__(self, n):
        super().__init__()
        self.n = 0
    def run(self):
        while self.n > 0:

            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)

c = CountdownThread(5)
c.start()

In [23]:
from threading import Thread, Event
import time

def countdown(n, started_evt):
    print('countdown starting')
    started_evt.set()
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(3)
        
started_evt = Event()

print('Launching countdown')
t = Thread(target=countdown, args=(10, started_evt))
t.start()
started_evt.wait()
print('countdown is running')

Launching countdown
countdown starting
T-minus 10
countdown is running
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


[理解Python中的线程](http://python.jobbole.com/85287/)

In [34]:
import time
import urllib.request

def get_responses():
    urls = [
       'http://www.baidu.com',
        'http://v3.bootcss.com/',
        'http://www.ebay.com',
        'http://www.alibaba.com',
        'http://www.codingpy.com/' 
    ]
    start = time.time()
    for url in urls:
        print(url)
        response = urllib.request.urlopen(url)
        print(response.getcode())
    print('Elapsed time: %s' %(time.time()-start))
get_responses()

http://www.baidu.com
200
http://v3.bootcss.com/
200
http://www.ebay.com
200
http://www.alibaba.com
200
http://www.codingpy.com/
200
Elapsed time: 2.4693312644958496


In [33]:
import time
import urllib.request
from threading import Thread

class GetUrlThread(Thread):
    def __init__(self, url):
        self.url = url
        super(GetUrlThread, self).__init__()
        
    def run(self):
        resp = urllib.request.urlopen(self.url)
        print(self.url, resp.getcode())
        

def get_responses():
    urls = [
       'http://www.baidu.com',
        'http://v3.bootcss.com/',
        'http://www.ebay.com',
        'http://www.alibaba.com',
        'http://www.codingpy.com/' 
    ]
    start = time.time()
    threads = []
    for url in urls:
        t = GetUrlThread(url)
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('Elapsed time: %s' %(time.time()-start))
get_responses()

http://www.baidu.com 200
http://www.alibaba.com 200
http://v3.bootcss.com/ 200
http://www.codingpy.com/ 200
http://www.ebay.com 200
Elapsed time: 1.564070463180542


In [18]:
from threading import Thread
import time

class MyThread(Thread):
    def run(self):
        for i in range(3):
            time.sleep(1)
            print('This is %s @ %d' %(self.name, i))
            
            
def test():
    threads = []
    for i in range(5):
        t = MyThread()
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('Ending Main threading')
        
test()

This is Thread-491 @ 0
This is Thread-492 @ 0
This is Thread-493 @ 0
This is Thread-494 @ 0
This is Thread-495 @ 0
This is Thread-491 @ 1
This is Thread-492 @ 1
This is Thread-493 @ 1
This is Thread-494 @ 1
This is Thread-495 @ 1
This is Thread-491 @ 2
This is Thread-493 @ 2
This is Thread-492 @ 2
This is Thread-494 @ 2
This is Thread-495 @ 2
Ending Main threading


In [22]:
from threading import Thread

# some_var = 0
class IncrementThread(Thread):
    def run(self):
        global some_var
#         read_value = some_var
#         print('some_var in %s is %d' %(self.name, read_value))
        time.sleep(1)
        read_value = some_var
        for i in range(100):
            some_var  += 1
        
#         print('read_var in %s is %d' %(self.name, read_value))
        
        print('some_var in %s  is %d' %(self.name, some_var))
        
def use_increment_thread():
    threads = []
    for i in range(10):
        t = IncrementThread()
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('After 50 modifications, some_var is %d' % (some_var,))
use_increment_thread()

some_var in Thread-607  is 1100some_var in Thread-608  is 1200
some_var in Thread-606  is 1300

some_var in Thread-611  is 1400
some_var in Thread-610  is 1500
some_var in Thread-609  is 1600
some_var in Thread-613  is 1700
some_var in Thread-612  is 1800
some_var in Thread-614  is 1900
some_var in Thread-615  is 2000
After 50 modifications, some_var is 2000


In [32]:
from threading import Thread
import time

class CreateListThread(Thread):
    def run(self):
        self.entries = []
        for i in range(10):
            time.sleep(1)
            self.entries.append(i)
        print(self.entries)

def use_create_list_thread():
    for i in range(3):
        t = CreateListThread()
        t.start()

use_create_list_thread()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9][0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



In [34]:
import threading
local = threading.local()
def func(name):
    print('current thread:%s' % threading.currentThread().name)
    local.name = name
    print("%s in %s" % (local.name,threading.currentThread().name))
t1 = threading.Thread(target=func,args=('haibo',))
t2 = threading.Thread(target=func,args=('lina',))
t1.start()
t2.start()
t1.join()
t2.join()

current thread:Thread-635
haibo in Thread-635
current thread:Thread-636
lina in Thread-636
